# BEE 4750 Lab 2: Uncertainty and Monte Carlo

**Name**:

**ID**:

> **Due Date**
>
> Wednesday, 9/25/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

In [1]:
using Random # random number generation
using Distributions # probability distributions and interface
using Statistics # basic statistical functions, including mean
using Plots # plotting

## Overview

In this lab, we will use Monte Carlo analysis to analyze a version of
the culmination of every episode of the long-running game show [The
Price Is Right](https://en.wikipedia.org/wiki/The_Price_Is_Right): the
[Showcase](https://en.wikipedia.org/wiki/The_Price_Is_Right#Showcase).

Monte Carlo methods involve the simulation of random numbers from
probability distributions. In an environmental context, we often
propagate these random numbers through some more complicated model and
then compute a resulting statistic which is relevant for assessing
performance or risk, such as an average outcome or a particular
quantile.

You should always start any computing with random numbers by setting a
“seed,” which controls the sequence of numbers which are generated
(since these are not *really* random, just “pseudorandom”). In Julia, we
do this with the `Random.seed!()` function.

In [1]:
Random.seed!(1)

TaskLocalRNG()

It doesn’t matter what seed you set, though different seeds might result
in slightly different values. But setting a seed means every time your
notebook is run, the answer will be the same.

> **Seeds and Reproducing Solutions**
>
> If you don’t re-run your code in the same order or if you re-run the
> same cell repeatedly, you will not get the same solution. If you’re
> working on a specific problem, you might want to re-use
> `Random.seed()` near any block of code you want to re-evaluate
> repeatedly.

### Probability Distributions and Julia

Julia provides a common interface for probability distributions with the
[`Distributions.jl`
package](https://juliastats.org/Distributions.jl/stable/). The basic
workflow for sampling from a distribution is:

1.  Set up the distribution. The specific syntax depends on the
    distribution and what parameters are required, but the general call
    is the similar. For a normal distribution or a uniform distribution,
    the syntax is

    ``` julia
    # you don't have to name this "normal_distribution"
    # μ is the mean and σ is the standard deviation
    normal_distribution = Normal(μ, σ)
    # a is the upper bound and b is the lower bound; these can be set to +Inf or -Inf for an unbounded distribution in one or both directions.
    uniform_distribution = Uniform(a, b)
    ```

    There are lots of both
    [univariate](https://juliastats.org/Distributions.jl/stable/univariate/#Index)
    and
    [multivariate](https://juliastats.org/Distributions.jl/stable/multivariate/)
    distributions, as well as the ability to create your own, but we
    won’t do anything too exotic here.

2.  Draw samples. This uses the `rand()` command (which, when used
    without a distribution, just samples uniformly from the interval
    $[0, 1]$.) For example, to sample from our normal distribution
    above:

    ``` julia
    # draw n samples
    rand(normal_distribution, n)
    ```

Putting this together, let’s say that we wanted to simulate 100
six-sided dice rolls. We could use a [Discrete Uniform
distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.DiscreteUniform).

In [1]:
dice_dist = DiscreteUniform(1, 6) # can generate any integer between 1 and 6
dice_rolls = rand(dice_dist, 100) # simulate rolls

100-element Vector{Int64}:
 3
 4
 1
 3
 3
 1
 3
 1
 3
 1
 ⋮
 5
 2
 1
 4
 2
 2
 6
 3
 1

And then we can plot a histogram of these rolls:

In [1]:
histogram(dice_rolls, legend=:false, bins=6)
ylabel!("Count")
xlabel!("Dice Value")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path clip-path="url(#clip480)" d="M699.099 1794.89 L715.418 1794.89 L715.418 1798.82 L693.474 1798.82 L693.474 1794.89 Q696.136 1792.13 700.719 1787.5 Q705.326 1782.85 706.506 1781.51 Q708.752 1778.98 709.631 1777.25 Q710.534 1775.49 710.534 1773.8 Q710.534 1771.04 708.59 1769.31 Q706.668 1767.57 703.566 1767.57 Q701.367 1767.57 698.914 1768.34 Q696.483 1769.1 693.705 1770.65 L693.705 1765.93 Q696.529 1764.79 698.983 1764.21 Q701.437 1763.64 703.474 1763.64 Q708.844 1763.64 712.039 1766.32 Q715.233 1769.01 715.233 1773.5 Q715.233 1775.63 714.423 1777.55 Q713.636 1779.45 711.529 1782.04 Q710.951 1782.71 707.849 1785.93 Q704.747 1789.12 699.099 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip480)" d="M1069.22 1780.19 Q1072.58 1780.9 1074.45 1783.17 Q1076.35 1785.44 1076.35 1788.77 Q1076.35 1793.89 1072.83 1796.69 Q1069.31 1799.49 1062.83 1799.49 Q1060.65 1799.49 1058.34 1799.05 Q1056.05 1798.64 1053.59 1797.78 L1053.59 1793.27 Q1055.54 1794.4 1057.85 1794.98 Q1060.17 1795.56 1062.69 1795.56 Q1067.09 1795.56 1069.38 1793.82 Q1071.7 1792.08 1071.7 1788.77 Q1071.7 1785.72 1069.54 1784.01 Q1067.41 1782.27 1063.59 1782.27 L1059.57 1782.27 L1059.57 1778.43 L1063.78 1778.43 Q1067.23 1778.43 1069.06 1777.06 Q1070.89 1775.67 1070.89 1773.08 Q1070.89 1770.42 1068.99 1769.01 Q1067.11 1767.57 1063.59 1767.57 Q1061.67 1767.57 1059.47 1767.99 Q1057.27 1768.4 1054.64 1769.28 L1054.64 1765.12 Q1057.3 1764.38 1059.61 1764.01 Q1061.95 1763.64 1064.01 1763.64 Q1069.33 1763.64 1072.44 1766.07 Q1075.54 1768.47 1075.54 1772.59 Q1075.54 1775.46 1073.9 1777.46 Q1072.25 1779.42 1069.22 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip480)" d="M1776.3 1764.26 L1794.66 1764.26 L1794.66 1768.2 L1780.58 1768.2 L1780.58 1776.67 Q1781.6 1776.32 1782.62 1776.16 Q1783.64 1775.97 1784.66 1775.97 Q1790.44 1775.97 1793.82 1779.15 Q1797.2 1782.32 1797.2 1787.73 Q1797.2 1793.31 1793.73 1796.41 Q1790.26 1799.49 1783.94 1799.49 Q1781.76 1799.49 1779.49 1799.12 Q1777.25 1798.75 1774.84 1798.01 L1774.84 1793.31 Q1776.93 1794.45 1779.15 1795 Q1781.37 1795.56 1783.85 1795.56 Q1787.85 1795.56 1790.19 1793.45 Q1792.53 1791.34 1792.53 1787.73 Q1792.53 1784.12 1790.19 1782.02 Q1787.85 1779.91 1783.85 1779.91 Q1781.97 1779.91 1780.1 1780.33 Q1778.24 1780.74 1776.3 1781.62 L1776.3 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip480)" d="M2146.95 1779.68 Q2143.8 1779.68 2141.95 1781.83 Q2140.12 1783.98 2140.12 1787.73 Q2140.12 1791.46 2141.95 1793.64 Q2143.8 1795.79 2146.95 1795.79 Q2150.1 1795.79 2151.93 1793.64 Q2153.78 1791.46 2153.78 1787.73 Q2153.78 1783.98 2151.93 1781.83 Q2150.1 1779.68 2146.95 1779.68 M2156.24 1765.02 L2156.24 1769.28 Q2154.48 1768.45 2152.67 1768.01 Q2150.89 1767.57 2149.13 1767.57 Q2144.5 1767.57 2142.05 1770.7 Q2139.61 1773.82 2139.27 1780.14 Q2140.63 1778.13 2142.69 1777.06 Q2144.75 1775.97 2147.23 1775.97 Q2152.44 1775.97 2155.45 1779.15 Q2158.48 1782.29 2158.48 1787.73 Q2158.48 1793.06 2155.33 1796.27 Q2152.18 1799.49 2146.95 1799.49 Q2140.96 1799.49 2137.79 1794.91 Q2134.61 1790.3 2134.61 1781.58 Q2134.61 1773.38 2138.5 1768.52 Q2142.39 1763.64 2148.94 1763.64 Q2150.7 1763.64 2152.49 1763.98 Q2154.29 1764.33 2156.24 1765.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip480)" d="M1352.15 1849.02 L1352.15 1854.5 Q1349.67 1853.13 1347.15 1852.46 Q1344.67 1851.76 1342.12 1851.76 Q1336.42 1851.76 1333.27 1855.39 Q1330.12 1858.98 1330.12 1865.51 Q1330.12 1872.03 1333.27 1875.66 Q1336.42 1879.26 1342.12 1879.26 Q1344.67 1879.26 1347.15 1878.59 Q1349.67 1877.89 1352.15 1876.52 L1352.15 1881.93 Q1349.7 1883.08 1347.06 1883.65 Q1344.45 1884.22 1341.49 1884.22 Q1333.43 1884.22 1328.69 1879.16 Q1323.95 1874.1 1323.95 1865.51 Q1323.95 1856.79 1328.72 1851.79 Q1333.53 1846.79 1341.87 1846.79 

### Instructions

**Remember to**:

-   Evaluate all of your code cells, in order (using a `Run All`
    command). This will make sure all output is visible and that the
    code cells were evaluated in the correct order.
-   Tag each of the problems when you submit to Gradescope; a 10%
    penalty will be deducted if this is not done.

## Exercise (3 Points)

The Showcase is the final round of every episode of The Price is Right,
matching the two big winners from the episode. Each contestant is shown
a “showcase” of prizes, which are usually some combination of a trip, a
motor vehicle, some furniture, and maybe some other stuff. They then
each have to make a bid on the retail price of the showcase. The rules
are:

-   an overbid is an automatic loss;
-   the contest who gets closest to the retail price wins their
    showcase;
-   if a contestant gets within \$250 of the retail price and is closer
    than their opponent, they win both showcases.

Your goal is to find a wager which maximizes your expected winnings,
which we may as well call utility, based on your assessment of the
probability of your showcase retail price. Make the following
assumptions about your expected winnings if you don’t overbid:

-   If you win both showcases, the value is the double of the single
    showcase value.
-   If you did not win both showcases but bid under the showcase value,
    the probability of being outbid increases linearly as the distance
    between your bid and the value increases (in other words, if you bid
    the exact value, you win with probability 1, and if you bid \$0, you
    win with probability 0).

We’ll assume that the distribution of all showcases offered by the show
is given as truncated normal distribution, which means a normal
distribution which has an upper and/or lower bound. `Distributions.jl`
makes it easy to specify truncations on any distribution, not just
normal distributions. For example, we’ll use the distribution shown in
<a href="#fig-values" class="quarto-xref">Figure 1</a> for the showcase
values (you can just directly use the `showcase_dist` distribution in
your lab code).

In [1]:
showcase_dist = truncated(Normal(31000, 4500), lower=5000, upper=42000)

Find your expected winnings if you bid \$35,000. Plot the outcomes of
your Monte Carlo experiment (iterations vs. running estimate). How did
you decide how many samples to use?

## References

Put any consulted sources here, including classmates you worked with/who
helped you.